<a href="https://colab.research.google.com/github/Redwolf0023/AutoMLvsBayesian/blob/main/WineQuality_GP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the libraries
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,ConstantKernel, DotProduct, Matern,RationalQuadratic
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc, log_loss, r2_score,mean_squared_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score as ras

In [ ]:
data = pd.read_csv('winequality-red.csv')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
x = data.drop('quality', axis=1).values
y = data['quality'].values

In [ ]:
from sklearn import preprocessing
SS = preprocessing.StandardScaler()
LE = preprocessing.LabelEncoder()
x = SS.fit_transform(x)
y = LE.fit_transform(y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize the Gaussian Process Regressor using Rational quadratic kernel
kernel = 1.0* RationalQuadratic(length_scale=1.0, alpha=1.0)
gpr = GaussianProcessRegressor(kernel=kernel, alpha=5e-9, optimizer='fmin_l_bfgs_b',
                                n_restarts_optimizer=0, normalize_y=False, copy_X_train=True,
                                random_state=20)

# Train the Gaussian Process Regressor
gpr.fit(x_train, y_train)

# Make predictions on the test set
predictions = gpr.predict(x_test)

In [ ]:
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(f"Mean Squared Error: {mse}")
r2 = r2_score(y_test, predictions)

# Print the calculated metrics
print(f"Root Mean Squared Error: {rmse}")
print(f"R2 Score: {r2}")


Mean Squared Error: 0.27015163553993987
Root Mean Squared Error: 0.5197611331563181
R2 Score: 0.5866117622903833


In [ ]:
# Calculate ROC curve and AUC for each class (one-vs-all)
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = len(np.unique(y))
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test == i, y_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Print evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"Precision (Macro-average): {precision_macro}")
print(f"Recall (Macro-average): {recall_macro}")
print(f"F1-Score (Macro-average): {f1_macro}")
print(f"Negative Log-Likelihood: {log_likelihood}")
print(f"Root mean squared error (Macro-average): {rmse}")
print(f"R squared error: {r2}")
print("Confusion Matrix:")
print(conf_matrix)

# Plot ROC curve for each class
plt.figure()
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], lw=2, label='ROC curve (class %d) (AUC = %0.2f)' % (i, roc_auc[i]))
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) - Multiclass')
plt.legend(loc='lower right')
plt.show()

NameError: ignored